In [28]:
import ForwardDiff as fd
import Zygote as zg
import LinearAlgebra as la

In [58]:
A = Matrix{Float64}( la.reshape(1:9, (3,3)) )

3×3 Matrix{Float64}:
 1.0  4.0  7.0
 2.0  5.0  8.0
 3.0  6.0  9.0

In [59]:
v = Vector{Float64}(1:3)

3-element Vector{Float64}:
 1.0
 2.0
 3.0

In [60]:
y = zeros(3)

3-element Vector{Float64}:
 0.0
 0.0
 0.0

In [61]:
function test!(y, A, v)
    return la.mul!(y, A, v)
end

test! (generic function with 1 method)

In [63]:
@time test!(y, A, v)

  0.000008 seconds


3-element Vector{Float64}:
 30.0
 36.0
 42.0

In [107]:
function sig(x)
    return 1 ./ (1 .+ exp.(-x))
end

sig (generic function with 1 method)

In [108]:
function sig_der!(delta, output)
    delta .= delta.* output .* (1 .- output)
end

sig_der! (generic function with 1 method)

In [148]:
x = rand(5)

W1 = rand(5, 5)
b1 = rand(5)

W2 = rand(2, 5)
b2 = rand(2)

W3 = rand(3, 2)
b3 = rand(3)

linear(x) = sum(sig(W3 * sig(W2 * sig(W1 * x + b1) .+ b2) .+ b3))

grads = zg.gradient(() -> sum(linear(x)), zg.Params([W2, b2, W1, b1, W3, b3]))

Grads(...)

In [149]:
delta = ones(3)

sig_der!(delta, sig(W3 * sig(W2 * sig(W1 * x + b1) .+ b2) .+ b3))

print(delta * transpose(sig(W2 * sig(W1 * x + b1) .+ b2)) ≈ grads[W3])
print(delta ≈ grads[b3])
delta = transpose(W3) * delta

sig_der!(delta, sig(W2 * sig(W1 * x + b1) .+ b2))

print(delta * transpose(sig(W1 * x + b1)) ≈ grads[W2])
print(delta ≈ grads[b2])
delta = transpose(W2) * delta

sig_der!(delta, sig(W1 * x + b1))

print(delta * transpose(x) ≈ grads[W1])
print(delta ≈ grads[b1])

truetruetruetruetruetrue

In [ ]:
1x5

## Kinetic

In [160]:
linear(x) = sum(sig(W1 * x + b1))

hess = la.tr(zg.hessian(linear, x))

-0.4607676558130107